# SHFE Silver (AG) Advanced Market Analysis
## Quantitative Framework for Options Trading

**Author:** Senior Quantitative Researcher  
**Target:** SHFE AG2604 Futures & Options Chain  
**Methodology:** SABR Calibration, Volatility Skew Analysis, Dealer Gamma Exposure (GEX)

---

### Analysis Framework

1. **Data Ingestion**: WindPy snapshot data for futures and full option chain
2. **Futures Analysis**: Basis structure and speculative interest metrics
3. **SABR Calibration**: Rigorous volatility surface modeling
4. **Market Microstructure**: Dealer positioning and gamma profiles
5. **Strategic Synthesis**: Actionable trading insights

In [ ]:
# Core Dependencies
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from typing import Optional, List, Dict, Tuple
from datetime import datetime, timedelta
import scipy.optimize as optimize
from scipy.stats import norm
from scipy.interpolate import CubicSpline, interp1d

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Wind API
from WindPy import w

# Styling
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ All dependencies loaded successfully")

## 1. Data Structures

Define typed data structures for robust data handling.

In [ ]:
@dataclass
class FuturesSnapshot:
    """SHFE Futures Contract Snapshot"""
    contract_code: str
    price: float
    bid: float
    ask: float
    volume: float
    open_interest: float
    timestamp: datetime
    
    @property
    def mid_price(self) -> float:
        return (self.bid + self.ask) / 2
    
    @property
    def spread_bps(self) -> float:
        """Bid-ask spread in basis points"""
        return 10000 * (self.ask - self.bid) / self.mid_price
    
    @property
    def volume_oi_ratio(self) -> float:
        """Speculative heat indicator"""
        return self.volume / self.open_interest if self.open_interest > 0 else 0


@dataclass
class OptionSnapshot:
    """SHFE Option Contract Snapshot"""
    contract_code: str
    option_type: str  # 'C' or 'P'
    strike: float
    price: float
    bid: float
    ask: float
    volume: float
    open_interest: float
    implied_vol: Optional[float]  # Wind's pre-calculated IV
    timestamp: datetime
    
    @property
    def mid_price(self) -> float:
        return (self.bid + self.ask) / 2 if self.bid > 0 and self.ask > 0 else self.price
    
    @property
    def is_liquid(self) -> bool:
        """Liquidity filter: Non-zero OI and reasonable bid-ask"""
        return self.open_interest > 0 and self.bid > 0 and self.ask > 0


@dataclass
class SABRParameters:
    """SABR Model Parameters"""
    alpha: float  # Initial volatility
    beta: float   # CEV exponent (0=Normal, 1=Lognormal)
    rho: float    # Correlation between asset and volatility
    nu: float     # Volatility of volatility
    f: float      # Forward price
    t: float      # Time to expiry
    
    def __post_init__(self):
        """Parameter constraints"""
        assert 0 <= self.beta <= 1, "Beta must be in [0, 1]"
        assert -1 < self.rho < 1, "Rho must be in (-1, 1)"
        assert self.alpha > 0 and self.nu > 0, "Alpha and Nu must be positive"


@dataclass
class GreeksProfile:
    """Option Greeks"""
    delta: float
    gamma: float
    vega: float
    theta: float


@dataclass
class GammaExposure:
    """Dealer Gamma Exposure per Strike"""
    strike: float
    call_oi: float
    put_oi: float
    call_gamma: float
    put_gamma: float
    net_gex: float  # Net Dealer GEX (negative = short gamma)
    
    @property
    def gex_notional(self) -> float:
        """GEX in notional terms (for visualization)"""
        return self.net_gex * self.strike


print("✓ Data structures defined")

## 2. WindPy Data Ingestion

Fetch snapshot data for AG2604 futures and the complete option chain.

In [ ]:
class WindDataFetcher:
    """Wind API Data Fetcher with Error Handling"""
    
    def __init__(self):
        self.w = w
        if not self.w.isconnected():
            self.w.start()
            print("✓ Wind API connected")
    
    def fetch_futures_snapshot(self, contract_code: str) -> FuturesSnapshot:
        """Fetch futures snapshot data"""
        fields = "rt_last,rt_bid,rt_ask,volume,oi"
        data = self.w.wsq(contract_code, fields)
        
        if data.ErrorCode != 0:
            raise ValueError(f"Wind API Error: {data.ErrorCode}")
        
        return FuturesSnapshot(
            contract_code=contract_code,
            price=data.Data[0][0],
            bid=data.Data[1][0],
            ask=data.Data[2][0],
            volume=data.Data[3][0],
            open_interest=data.Data[4][0],
            timestamp=datetime.now()
        )
    
    def fetch_option_chain(self, underlying_code: str) -> List[OptionSnapshot]:
        """Fetch full option chain for a futures contract"""
        # Get option chain codes
        chain_data = self.w.wset(
            "optionchain",
            f"us_code={underlying_code};option_var=全部;call_put=全部"
        )
        
        if chain_data.ErrorCode != 0:
            raise ValueError(f"Option chain fetch failed: {chain_data.ErrorCode}")
        
        option_codes = chain_data.Data[0]
        strikes = chain_data.Data[1]
        call_puts = chain_data.Data[2]
        
        # Fetch market data for all options
        fields = "rt_last,rt_bid,rt_ask,volume,oi,impliedvolatility"
        market_data = self.w.wsq(option_codes, fields)
        
        options = []
        for i, code in enumerate(option_codes):
            try:
                option = OptionSnapshot(
                    contract_code=code,
                    option_type='C' if call_puts[i] == '认购' else 'P',
                    strike=strikes[i],
                    price=market_data.Data[0][i] if market_data.Data[0][i] else 0,
                    bid=market_data.Data[1][i] if market_data.Data[1][i] else 0,
                    ask=market_data.Data[2][i] if market_data.Data[2][i] else 0,
                    volume=market_data.Data[3][i] if market_data.Data[3][i] else 0,
                    open_interest=market_data.Data[4][i] if market_data.Data[4][i] else 0,
                    implied_vol=market_data.Data[5][i] if market_data.Data[5][i] else None,
                    timestamp=datetime.now()
                )
                options.append(option)
            except Exception as e:
                print(f"Warning: Failed to parse {code}: {e}")
                continue
        
        return options
    
    def close(self):
        """Close Wind connection"""
        self.w.close()


# Initialize fetcher
fetcher = WindDataFetcher()
print("✓ Wind data fetcher initialized")

In [ ]:
# Fetch data for AG2604
UNDERLYING = "AG2604.SHF"

try:
    # Futures snapshot
    futures = fetcher.fetch_futures_snapshot(UNDERLYING)
    print(f"\n{'='*60}")
    print(f"FUTURES SNAPSHOT: {UNDERLYING}")
    print(f"{'='*60}")
    print(f"Price:          {futures.price:,.2f}")
    print(f"Bid/Ask:        {futures.bid:,.2f} / {futures.ask:,.2f}")
    print(f"Spread:         {futures.spread_bps:.1f} bps")
    print(f"Volume:         {futures.volume:,.0f}")
    print(f"Open Interest:  {futures.open_interest:,.0f}")
    print(f"Vol/OI Ratio:   {futures.volume_oi_ratio:.2%}")
    
    # Option chain
    options = fetcher.fetch_option_chain(UNDERLYING)
    liquid_options = [opt for opt in options if opt.is_liquid]
    
    print(f"\nOPTION CHAIN")
    print(f"Total Contracts:  {len(options)}")
    print(f"Liquid Contracts: {len(liquid_options)}")
    
    # Convert to DataFrame for analysis
    options_df = pd.DataFrame([
        {
            'strike': opt.strike,
            'type': opt.option_type,
            'price': opt.mid_price,
            'oi': opt.open_interest,
            'volume': opt.volume,
            'iv': opt.implied_vol
        }
        for opt in liquid_options
    ])
    
    print(f"\nStrike Range: {options_df['strike'].min():.0f} - {options_df['strike'].max():.0f}")
    print(f"ATM Strike (approx): {options_df.iloc[(options_df['strike'] - futures.price).abs().argsort()[:1]]['strike'].values[0]:.0f}")
    
except Exception as e:
    print(f"ERROR: Data fetch failed - {e}")
    print("\nNote: This notebook requires live Wind API access.")
    print("For demonstration, we'll generate synthetic data...\n")
    
    # Fallback: Synthetic data for testing
    futures = FuturesSnapshot(
        contract_code=UNDERLYING,
        price=6500.0,
        bid=6498.0,
        ask=6502.0,
        volume=125000,
        open_interest=580000,
        timestamp=datetime.now()
    )
    
    # Generate synthetic option chain
    np.random.seed(42)
    strikes = np.arange(6000, 7100, 100)
    options = []
    
    for strike in strikes:
        moneyness = strike / futures.price
        # Realistic skew: higher IV for OTM puts
        base_iv = 0.25
        iv = base_iv + 0.05 * (1 - moneyness) if moneyness < 1 else base_iv - 0.02 * (moneyness - 1)
        iv = max(0.15, min(0.40, iv + np.random.normal(0, 0.01)))
        
        for opt_type in ['C', 'P']:
            oi = np.random.lognormal(8, 1.5) * (1.5 if abs(moneyness - 1) < 0.05 else 1)
            options.append(OptionSnapshot(
                contract_code=f"{UNDERLYING}_{opt_type}_{strike}",
                option_type=opt_type,
                strike=strike,
                price=max(10, np.random.lognormal(4, 1)),
                bid=100,
                ask=110,
                volume=np.random.lognormal(5, 1.2),
                open_interest=oi,
                implied_vol=iv,
                timestamp=datetime.now()
            ))
    
    liquid_options = options
    options_df = pd.DataFrame([
        {
            'strike': opt.strike,
            'type': opt.option_type,
            'price': opt.mid_price,
            'oi': opt.open_interest,
            'volume': opt.volume,
            'iv': opt.implied_vol
        }
        for opt in liquid_options
    ])
    
    print("✓ Synthetic data generated for demonstration")

## 3. Futures Market Analysis

### 3.1 Speculative Activity Metrics

In [ ]:
# Volume/OI analysis
print(f"\n{'='*60}")
print("FUTURES MARKET MICROSTRUCTURE")
print(f"{'='*60}\n")

print(f"Spot Price:       {futures.price:,.2f}")
print(f"Volume/OI Ratio:  {futures.volume_oi_ratio:.2%}")

# Interpretation
if futures.volume_oi_ratio > 0.5:
    activity = "HIGH speculative churn (intraday momentum traders)"
elif futures.volume_oi_ratio > 0.2:
    activity = "MODERATE activity (balanced positioning)"
else:
    activity = "LOW turnover (long-term holders dominate)"

print(f"\n🔍 Interpretation: {activity}")
print(f"   Spread: {futures.spread_bps:.1f} bps → {'Liquid' if futures.spread_bps < 10 else 'Wide (caution)'}")

## 4. SABR Model Implementation

### 4.1 Mathematical Foundation

The SABR (Stochastic Alpha Beta Rho) model by Hagan et al. describes the evolution of forward prices and volatility:

$$
\begin{align}
dF_t &= \sigma_t F_t^\beta dW_t^F \\
d\sigma_t &= \nu \sigma_t dW_t^\sigma \\
dW_t^F dW_t^\sigma &= \rho dt
\end{align}
$$

The implied volatility approximation for strike $K$:

$$
\sigma_{IV}(K, T) \approx \frac{\alpha}{(FK)^{(1-\beta)/2}[1 + \frac{(1-\beta)^2}{24}\log^2(F/K) + \ldots]} \times \frac{z}{x(z)}
$$

where $z = \frac{\nu}{\alpha}(FK)^{(1-\beta)/2}\log(F/K)$ and $x(z) = \log\left(\frac{\sqrt{1-2\rho z + z^2} + z - \rho}{1-\rho}\right)$

In [ ]:
class SABRModel:
    """SABR Volatility Model with Hagan's Formula"""
    
    @staticmethod
    def sabr_volatility(strike: float, params: SABRParameters) -> float:
        """
        Hagan et al. SABR implied volatility formula.
        
        Returns:
            Implied volatility (in decimal form, e.g., 0.25 for 25%)
        """
        F, K, T = params.f, strike, params.t
        alpha, beta, rho, nu = params.alpha, params.beta, params.rho, params.nu
        
        # ATM case
        if abs(F - K) < 1e-6:
            term1 = alpha / (F ** (1 - beta))
            term2 = 1 + T * (
                ((1 - beta) ** 2 / 24) * (alpha ** 2 / (F ** (2 - 2 * beta)))
                + 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))
                + (2 - 3 * rho ** 2) * nu ** 2 / 24
            )
            return term1 * term2
        
        # General case
        log_FK = np.log(F / K)
        FK_mid = (F * K) ** ((1 - beta) / 2)
        
        # z parameter
        z = (nu / alpha) * FK_mid * log_FK
        
        # x(z) with numerical stability
        if abs(z) < 1e-7:
            x_z = 1  # Taylor expansion: x(z) ≈ 1 when z → 0
        else:
            sqrt_term = np.sqrt(1 - 2 * rho * z + z ** 2)
            numerator = sqrt_term + z - rho
            denominator = 1 - rho
            x_z = np.log(numerator / denominator) / z
        
        # Main formula
        numerator = alpha
        denominator = FK_mid * (1 + ((1 - beta) ** 2 / 24) * log_FK ** 2
                                + ((1 - beta) ** 4 / 1920) * log_FK ** 4)
        
        vol_term = numerator / denominator * x_z
        
        # Time-dependent correction
        correction = 1 + T * (
            ((1 - beta) ** 2 / 24) * (alpha ** 2 / FK_mid ** 2)
            + 0.25 * rho * beta * nu * alpha / FK_mid
            + (2 - 3 * rho ** 2) * nu ** 2 / 24
        )
        
        return vol_term * correction
    
    @staticmethod
    def calibrate(
        strikes: np.ndarray,
        market_vols: np.ndarray,
        forward: float,
        time_to_expiry: float,
        beta: float = 0.7  # Common for commodities
    ) -> SABRParameters:
        """
        Calibrate SABR parameters to market implied volatilities.
        
        Optimization objective: Minimize sum of squared volatility errors.
        """
        # Initial guess
        atm_vol = np.median(market_vols)
        x0 = np.array([atm_vol, 0.0, 0.3])  # [alpha, rho, nu]
        
        def objective(x):
            alpha, rho, nu = x
            # Constraints within bounds
            if alpha <= 0 or nu <= 0 or abs(rho) >= 1:
                return 1e10
            
            try:
                params = SABRParameters(
                    alpha=alpha,
                    beta=beta,
                    rho=rho,
                    nu=nu,
                    f=forward,
                    t=time_to_expiry
                )
                
                model_vols = np.array([
                    SABRModel.sabr_volatility(K, params) for K in strikes
                ])
                
                # Sum of squared errors
                return np.sum((model_vols - market_vols) ** 2)
            except:
                return 1e10
        
        # Bounds: alpha > 0, -1 < rho < 1, nu > 0
        bounds = [(0.001, 2.0), (-0.99, 0.99), (0.001, 2.0)]
        
        result = optimize.minimize(
            objective,
            x0,
            method='L-BFGS-B',
            bounds=bounds
        )
        
        alpha_opt, rho_opt, nu_opt = result.x
        
        return SABRParameters(
            alpha=alpha_opt,
            beta=beta,
            rho=rho_opt,
            nu=nu_opt,
            f=forward,
            t=time_to_expiry
        )

print("✓ SABR model implementation complete")

### 4.2 Calibration to Market Data

In [ ]:
# Prepare calibration data
calls_df = options_df[options_df['type'] == 'C'].copy()
puts_df = options_df[options_df['type'] == 'P'].copy()

# Use liquid ATM/OTM options for calibration
atm_strike = calls_df.iloc[(calls_df['strike'] - futures.price).abs().argsort()[:1]]['strike'].values[0]
calibration_range = (futures.price * 0.90, futures.price * 1.10)

calib_calls = calls_df[
    (calls_df['strike'] >= calibration_range[0]) &
    (calls_df['strike'] <= calibration_range[1]) &
    (calls_df['iv'].notna())
].sort_values('strike')

# Time to expiry (assuming 1 month for AG2604 as of now)
# In production, calculate from contract maturity
T = 60 / 365  # Approximately 2 months

# Calibrate
print("\n🔧 Calibrating SABR model...\n")
sabr_params = SABRModel.calibrate(
    strikes=calib_calls['strike'].values,
    market_vols=calib_calls['iv'].values,
    forward=futures.price,
    time_to_expiry=T,
    beta=0.7  # Commodity standard
)

print(f"{'='*60}")
print("SABR CALIBRATION RESULTS")
print(f"{'='*60}\n")
print(f"α (alpha):  {sabr_params.alpha:.4f}  ← Initial volatility level")
print(f"β (beta):   {sabr_params.beta:.4f}  ← CEV exponent (fixed)")
print(f"ρ (rho):    {sabr_params.rho:+.4f} ← Asset-Vol correlation")
print(f"ν (nu):     {sabr_params.nu:.4f}  ← Vol-of-vol")
print(f"\nForward:    {sabr_params.f:.2f}")
print(f"Time:       {sabr_params.t:.4f} years\n")

# Model fit quality
model_vols = [SABRModel.sabr_volatility(K, sabr_params) for K in calib_calls['strike'].values]
rmse = np.sqrt(np.mean((np.array(model_vols) - calib_calls['iv'].values) ** 2))
print(f"Calibration RMSE: {rmse:.4f} ({rmse*100:.2f}% vol points)")

### 4.3 Volatility Smile Visualization

In [ ]:
# Generate smooth SABR surface
strike_range = np.linspace(
    futures.price * 0.85,
    futures.price * 1.15,
    100
)
sabr_vols = [SABRModel.sabr_volatility(K, sabr_params) for K in strike_range]

# Plot
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Volatility Smile', 'Model Fit Quality'),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}]]
)

# Left: Smile
fig.add_trace(
    go.Scatter(
        x=strike_range,
        y=np.array(sabr_vols) * 100,
        mode='lines',
        name='SABR Model',
        line=dict(color='#FF6B6B', width=3)
    ),
    row=1, col=1
)

# Market IVs
for opt_type, color, symbol in [('C', '#4ECDC4', 'circle'), ('P', '#95E1D3', 'diamond')]:
    subset = options_df[(options_df['type'] == opt_type) & (options_df['iv'].notna())]
    fig.add_trace(
        go.Scatter(
            x=subset['strike'],
            y=subset['iv'] * 100,
            mode='markers',
            name=f'Market {opt_type}',
            marker=dict(size=8, color=color, symbol=symbol, line=dict(width=1, color='white'))
        ),
        row=1, col=1
    )

# ATM line
fig.add_vline(
    x=futures.price,
    line_dash="dash",
    line_color="gray",
    annotation_text="ATM",
    row=1, col=1
)

# Right: Calibration quality
fig.add_trace(
    go.Scatter(
        x=calib_calls['iv'] * 100,
        y=np.array(model_vols) * 100,
        mode='markers',
        name='Model vs Market',
        marker=dict(size=10, color='#FFE66D', line=dict(width=1, color='#333'))
    ),
    row=1, col=2
)

# 45-degree line
min_iv, max_iv = min(calib_calls['iv']) * 100, max(calib_calls['iv']) * 100
fig.add_trace(
    go.Scatter(
        x=[min_iv, max_iv],
        y=[min_iv, max_iv],
        mode='lines',
        name='Perfect Fit',
        line=dict(color='gray', dash='dot')
    ),
    row=1, col=2
)

# Layout
fig.update_xaxes(title_text="Strike", row=1, col=1)
fig.update_yaxes(title_text="Implied Volatility (%)", row=1, col=1)
fig.update_xaxes(title_text="Market IV (%)", row=1, col=2)
fig.update_yaxes(title_text="SABR IV (%)", row=1, col=2)

fig.update_layout(
    title=f"SHFE AG2604 Volatility Surface (ρ={sabr_params.rho:+.3f})",
    template='plotly_white',
    height=500,
    showlegend=True,
    hovermode='x unified'
)

fig.show()

### 4.4 Risk Reversal & Butterfly Analysis

**Risk Reversal (25Δ):** $RR = IV_{Call,25\Delta} - IV_{Put,25\Delta}$  
**Butterfly (25Δ):** $BF = \frac{IV_{Call,25\Delta} + IV_{Put,25\Delta}}{2} - IV_{ATM}$

In [ ]:
# Black-Scholes Delta (simplified for strike approximation)
def bs_delta(strike, spot, vol, T, opt_type='C'):
    """Approximate BS delta for strike identification"""
    d1 = (np.log(spot / strike) + 0.5 * vol**2 * T) / (vol * np.sqrt(T))
    if opt_type == 'C':
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1

# Find 25-delta strikes (approximation)
atm_iv = SABRModel.sabr_volatility(futures.price, sabr_params)

# Search for strikes closest to 25-delta
target_delta = 0.25
call_deltas = [(K, abs(bs_delta(K, futures.price, atm_iv, T, 'C') - target_delta)) 
               for K in calls_df['strike'].values]
put_deltas = [(K, abs(bs_delta(K, futures.price, atm_iv, T, 'P') + target_delta)) 
              for K in puts_df['strike'].values]

strike_25d_call = min(call_deltas, key=lambda x: x[1])[0]
strike_25d_put = min(put_deltas, key=lambda x: x[1])[0]

# Calculate metrics
iv_25d_call = SABRModel.sabr_volatility(strike_25d_call, sabr_params)
iv_25d_put = SABRModel.sabr_volatility(strike_25d_put, sabr_params)
iv_atm = atm_iv

risk_reversal = (iv_25d_call - iv_25d_put) * 100  # In vol points
butterfly = ((iv_25d_call + iv_25d_put) / 2 - iv_atm) * 100

print(f"\n{'='*60}")
print("VOLATILITY SKEW METRICS")
print(f"{'='*60}\n")
print(f"ATM Strike:        {futures.price:.0f}")
print(f"ATM IV:            {iv_atm*100:.2f}%\n")
print(f"25Δ Call Strike:   {strike_25d_call:.0f}")
print(f"25Δ Call IV:       {iv_25d_call*100:.2f}%\n")
print(f"25Δ Put Strike:    {strike_25d_put:.0f}")
print(f"25Δ Put IV:        {iv_25d_put*100:.2f}%\n")
print(f"{'─'*60}")
print(f"Risk Reversal:     {risk_reversal:+.2f} vol pts")
print(f"Butterfly:         {butterfly:+.2f} vol pts")
print(f"{'─'*60}\n")

# Interpretation
print("🎯 INTERPRETATION:\n")

if risk_reversal < -1:
    print(f"   📉 NEGATIVE RR ({risk_reversal:.1f}) → PUT SKEW DOMINANT")
    print("      Market is pricing downside crash protection.")
    print("      Implies: Bearish sentiment, tail-risk hedging demand.\n")
elif risk_reversal > 1:
    print(f"   📈 POSITIVE RR ({risk_reversal:.1f}) → CALL SKEW DOMINANT")
    print("      Market is pricing upside breakout potential.")
    print("      Implies: Bullish sentiment, rally protection.\n")
else:
    print(f"   ⚖️  NEUTRAL RR ({risk_reversal:.1f}) → BALANCED SKEW")
    print("      No directional bias in volatility pricing.\n")

if butterfly > 1:
    print(f"   🦋 POSITIVE BF ({butterfly:.1f}) → CONVEXITY DEMAND")
    print("      Wings (OTM options) expensive vs ATM.")
    print("      Implies: Jump/crash risk premium, fat tails expected.\n")
else:
    print(f"   🦋 LOW BF ({butterfly:.1f}) → NORMAL DISTRIBUTION")
    print("      Volatility smile is relatively flat.\n")

print(f"   ρ (SABR correlation): {sabr_params.rho:+.3f}")
if sabr_params.rho < -0.3:
    print("      Strong negative correlation → Leverage effect dominant.")
    print("      Price drops amplify volatility (typical equity/commodity).")
elif sabr_params.rho > 0.3:
    print("      Positive correlation → Volatility rises WITH price.")
    print("      Unusual for commodities; suggests supply shock fears.")
else:
    print("      Near-zero correlation → Volatility independent of price direction.")

## 5. Dealer Gamma Exposure (GEX) Analysis

### 5.1 Greek Calculations

In [ ]:
class BSMGreeks:
    """Black-Scholes-Merton Greeks Calculator"""
    
    @staticmethod
    def d1(S, K, T, r, sigma):
        return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    @staticmethod
    def d2(S, K, T, r, sigma):
        return BSMGreeks.d1(S, K, T, r, sigma) - sigma * np.sqrt(T)
    
    @staticmethod
    def gamma(S, K, T, r, sigma):
        """Gamma: ∂²V/∂S²"""
        d1_val = BSMGreeks.d1(S, K, T, r, sigma)
        return norm.pdf(d1_val) / (S * sigma * np.sqrt(T))
    
    @staticmethod
    def delta(S, K, T, r, sigma, opt_type='C'):
        """Delta: ∂V/∂S"""
        d1_val = BSMGreeks.d1(S, K, T, r, sigma)
        if opt_type == 'C':
            return norm.cdf(d1_val)
        else:
            return norm.cdf(d1_val) - 1

# Calculate GEX for each strike
r = 0.02  # Risk-free rate (approximate)
gex_profiles = []

for strike in sorted(options_df['strike'].unique()):
    call_row = calls_df[calls_df['strike'] == strike]
    put_row = puts_df[puts_df['strike'] == strike]
    
    if call_row.empty or put_row.empty:
        continue
    
    # Use SABR vol for consistency
    sigma = SABRModel.sabr_volatility(strike, sabr_params)
    
    # Gamma (same for calls/puts)
    gamma_val = BSMGreeks.gamma(futures.price, strike, T, r, sigma)
    
    # Open Interest
    call_oi = call_row['oi'].values[0]
    put_oi = put_row['oi'].values[0]
    
    # Dealer GEX = -Client GEX (dealers short options to clients)
    # Positive GEX → Dealers long gamma (stabilizing)
    # Negative GEX → Dealers short gamma (destabilizing)
    call_gex = -call_oi * gamma_val * 100  # Scale by contract size
    put_gex = -put_oi * gamma_val * 100
    net_gex = call_gex + put_gex
    
    gex_profiles.append(GammaExposure(
        strike=strike,
        call_oi=call_oi,
        put_oi=put_oi,
        call_gamma=call_gex,
        put_gamma=put_gex,
        net_gex=net_gex
    ))

gex_df = pd.DataFrame([{
    'strike': g.strike,
    'call_oi': g.call_oi,
    'put_oi': g.put_oi,
    'net_gex': g.net_gex,
    'gex_notional': g.gex_notional
} for g in gex_profiles])

print("✓ Gamma exposure calculated")

### 5.2 GEX Profile Visualization

In [ ]:
# Identify key levels
zero_gamma_strikes = gex_df[gex_df['net_gex'].abs() < gex_df['net_gex'].abs().quantile(0.1)]['strike'].values
max_negative_gex_strike = gex_df.loc[gex_df['net_gex'].idxmin(), 'strike']
max_positive_gex_strike = gex_df.loc[gex_df['net_gex'].idxmax(), 'strike']

# Plot
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Dealer Net Gamma Exposure by Strike', 'Open Interest Distribution'),
    row_heights=[0.6, 0.4],
    vertical_spacing=0.12
)

# GEX bars
colors = ['#FF6B6B' if x < 0 else '#4ECDC4' for x in gex_df['net_gex']]
fig.add_trace(
    go.Bar(
        x=gex_df['strike'],
        y=gex_df['net_gex'],
        marker_color=colors,
        name='Net GEX',
        hovertemplate='Strike: %{x}<br>GEX: %{y:.0f}<extra></extra>'
    ),
    row=1, col=1
)

# Zero line
fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=1)

# Current spot
fig.add_vline(
    x=futures.price,
    line_dash="solid",
    line_color="gold",
    line_width=3,
    annotation_text=f"Spot: {futures.price:.0f}",
    annotation_position="top",
    row=1, col=1
)

# Open Interest
fig.add_trace(
    go.Bar(
        x=calls_df['strike'],
        y=calls_df['oi'],
        name='Call OI',
        marker_color='#95E1D3',
        opacity=0.7
    ),
    row=2, col=1
)
fig.add_trace(
    go.Bar(
        x=puts_df['strike'],
        y=puts_df['oi'],
        name='Put OI',
        marker_color='#FFE66D',
        opacity=0.7
    ),
    row=2, col=1
)

# Layout
fig.update_xaxes(title_text="Strike", row=2, col=1)
fig.update_yaxes(title_text="Net GEX (Dealer Perspective)", row=1, col=1)
fig.update_yaxes(title_text="Open Interest", row=2, col=1)

fig.update_layout(
    title="SHFE AG2604 Gamma Exposure Landscape",
    template='plotly_white',
    height=800,
    showlegend=True,
    hovermode='x unified'
)

fig.show()

# Summary
print(f"\n{'='*60}")
print("DEALER GAMMA EXPOSURE SUMMARY")
print(f"{'='*60}\n")
print(f"Current Spot:           {futures.price:.0f}\n")
print(f"Max Negative GEX:       {max_negative_gex_strike:.0f} (Dealers SHORT Gamma)")
print(f"   → Magnitude:         {gex_df['net_gex'].min():.0f}")
print(f"\nMax Positive GEX:       {max_positive_gex_strike:.0f} (Dealers LONG Gamma)")
print(f"   → Magnitude:         {gex_df['net_gex'].max():.0f}\n")

if len(zero_gamma_strikes) > 0:
    closest_zero = zero_gamma_strikes[np.argmin(np.abs(zero_gamma_strikes - futures.price))]
    print(f"Zero Gamma Level:       ~{closest_zero:.0f}")
    print(f"Distance from Spot:     {abs(closest_zero - futures.price):.0f} ({abs(closest_zero - futures.price)/futures.price*100:.1f}%)")

### 5.3 GEX Interpretation

**Key Concepts:**

1. **Negative GEX (Dealers Short Gamma):**
   - Dealers must buy as price rises, sell as price falls
   - **Amplifies volatility** (positive feedback loop)
   - Price tends to "repel" from these strikes

2. **Positive GEX (Dealers Long Gamma):**
   - Dealers sell into rallies, buy into dips
   - **Dampens volatility** (mean reversion)
   - Price tends to be "attracted" to these strikes ("Gamma Wall")

3. **Zero Gamma Level:**
   - Transition point between regimes
   - Break above/below can trigger regime shift

In [ ]:
# Calculate regime
current_gex = gex_df.iloc[(gex_df['strike'] - futures.price).abs().argsort()[:1]]['net_gex'].values[0]

print(f"\n{'='*60}")
print("VOLATILITY REGIME ANALYSIS")
print(f"{'='*60}\n")

if current_gex < -100:
    regime = "⚠️  NEGATIVE GAMMA ZONE (High Volatility Risk)"
    interpretation = """   
   Dealers are SHORT gamma around current price.
   → Expect amplified price swings
   → Trending behavior likely
   → Volatility accelerates on directional moves
   
   Trading Implications:
   - Long straddles/strangles benefit from vol expansion
   - Momentum strategies favored
   - Stop-losses critical (gaps possible)
   """
elif current_gex > 100:
    regime = "🛡️  POSITIVE GAMMA ZONE (Volatility Suppression)"
    interpretation = """   
   Dealers are LONG gamma around current price.
   → Price compression expected
   → Mean reversion forces active
   → Realized vol likely below implied
   
   Trading Implications:
   - Short premium strategies (iron condors)
   - Range-bound scalping
   - Fading extremes
   """
else:
    regime = "⚖️  NEUTRAL GAMMA (Transitional)"
    interpretation = """   
   Price near zero-gamma inflection point.
   → Regime shift possible
   → Monitor for breakout direction
   → Volatility regime unclear
   
   Trading Implications:
   - Wait for regime confirmation
   - Options cheap if expecting shift
   - Directional bets risky
   """

print(regime)
print(interpretation)

## 6. Trader's Note: Strategic Synthesis

---

### Executive Summary

In [ ]:
# Generate dynamic trading note
print(f"""
╔{'='*78}╗
║{' '*20}SHFE AG2604 MARKET INTELLIGENCE REPORT{' '*20}║
║{' '*30}{datetime.now().strftime('%Y-%m-%d %H:%M')}{' '*30}║
╚{'='*78}╝

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. MARKET SNAPSHOT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   Futures Price:       {futures.price:>10,.2f}
   ATM Implied Vol:     {iv_atm*100:>10.2f}%
   Volume/OI Ratio:     {futures.volume_oi_ratio:>10.1%}
   Market Activity:     {activity}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
2. VOLATILITY SURFACE DIAGNOSTICS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   SABR Parameters:
   ├─ α (Initial Vol):       {sabr_params.alpha:.4f}
   ├─ β (CEV Exponent):      {sabr_params.beta:.4f}
   ├─ ρ (Correlation):       {sabr_params.rho:+.4f}
   └─ ν (Vol-of-Vol):        {sabr_params.nu:.4f}

   Skew Metrics:
   ├─ Risk Reversal (25Δ):   {risk_reversal:+.2f} vol pts
   └─ Butterfly (25Δ):       {butterfly:+.2f} vol pts

   📊 Skew Interpretation:
""")

if risk_reversal < -1:
    print(f"""   
      ⚠️  PUT SKEW DOMINANT (RR = {risk_reversal:.1f})
      The market is pricing significant downside tail risk.
      OTM puts expensive relative to calls → Crash hedging demand.
      
      Negative ρ = {sabr_params.rho:.3f} confirms leverage effect:
      Volatility SURGES on price drops (classic commodity fear pattern).
   """)
elif risk_reversal > 1:
    print(f"""   
      🚀 CALL SKEW DOMINANT (RR = {risk_reversal:.1f})
      The market is pricing upside breakout potential.
      OTM calls expensive → Bullish positioning, rally protection.
      
      Positive ρ = {sabr_params.rho:.3f} suggests supply shock fears:
      Volatility rises WITH price (inverse of normal commodity behavior).
   """)
else:
    print(f"""   
      ⚖️  BALANCED SKEW (RR = {risk_reversal:.1f})
      No strong directional bias in volatility premium.
      ρ = {sabr_params.rho:.3f} → Moderate correlation regime.
   """)

print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
3. DEALER GAMMA POSITIONING
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   Current Regime:      {regime.split('(')[0].strip()}
   
   Key Strikes:
   ├─ Max Negative GEX:      {max_negative_gex_strike:>10,.0f} (Volatility Accelerator)
   ├─ Max Positive GEX:      {max_positive_gex_strike:>10,.0f} (Gamma Wall / Pin Risk)
   └─ Zero Gamma:            ~{closest_zero if len(zero_gamma_strikes) > 0 else 'N/A':>9} (Regime Flip Level)

   🎯 Positioning Insight:
""")

if current_gex < -100:
    print(f"""   
      Dealers are SHORT GAMMA near spot ({futures.price:.0f}).
      → Market makers must CHASE price (buy high, sell low).
      → Creates positive feedback loop → Amplifies trends.
      → Expect higher realized volatility than implied.
      
      ⚡ If spot breaks below {max_negative_gex_strike:.0f}:
         Dealers forced to dump hedges → Cascade risk.
   """)
elif current_gex > 100:
    print(f"""   
      Dealers are LONG GAMMA near spot ({futures.price:.0f}).
      → Market makers provide liquidity (sell high, buy low).
      → Creates mean reversion → Price compression.
      → Gamma Wall at {max_positive_gex_strike:.0f} acts as magnet.
      
      🛡️ Price likely to grind toward {max_positive_gex_strike:.0f} (pin risk at expiry).
   """)
else:
    print(f"""   
      Spot near ZERO GAMMA INFLECTION.
      → Volatility regime UNCLEAR.
      → Breakout in either direction could trigger regime shift.
      → Monitor closely for confirmation.
   """)

print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
4. ACTIONABLE TRADING STRATEGIES
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

# Strategy 1: Directional based on skew
if risk_reversal < -2 and current_gex < -100:
    print(f"""   
   📉 STRATEGY 1: Bearish Volatility Expansion Play
   
      Setup: High put skew + Negative GEX → Crash risk priced in.
      
      Trade:
      • BUY OTM Put Spread ({strike_25d_put:.0f}/{strike_25d_put - 200:.0f})
      • Target: Break below {max_negative_gex_strike:.0f} triggers dealer hedge selling.
      • Risk: Premium decay if market grinds sideways.
      
      Rationale:
      Negative gamma zone + elevated put IV = Tail risk not fully priced.
      If spot breaches {max_negative_gex_strike:.0f}, expect non-linear acceleration.
   """)
elif risk_reversal > 1 and current_gex > 100:
    print(f"""   
   📈 STRATEGY 1: Bullish Mean Reversion
   
      Setup: Call skew + Positive GEX → Upside capped by gamma wall.
      
      Trade:
      • SELL OTM Call Spread ({strike_25d_call:.0f}/{strike_25d_call + 200:.0f})
      • Target: Price pins toward {max_positive_gex_strike:.0f} into expiry.
      • Risk: Breakout above {max_positive_gex_strike:.0f} → Losses capped but painful.
      
      Rationale:
      Dealers long gamma = selling into rallies. Price magnet effect.
   """)
else:
    print(f"""   
   ⚖️  STRATEGY 1: Volatility Straddle
   
      Setup: Balanced skew + Uncertain regime → Bet on movement, not direction.
      
      Trade:
      • BUY ATM Straddle (Strike: {futures.price:.0f})
      • Target: Breakout from zero-gamma level → Vol expansion in either direction.
      • Risk: Theta decay if range-bound.
      
      Rationale:
      Regime uncertainty = mispriced realized vol. Straddle benefits from break.
   """)

# Strategy 2: Vol arb
if butterfly > 2:
    print(f"""   
   🦋 STRATEGY 2: Butterfly Spread (Sell Convexity)
   
      Setup: High BF ({butterfly:.1f}) → Wings overpriced vs ATM.
      
      Trade:
      • SELL Iron Butterfly (Short {futures.price:.0f} straddle, Long {strike_25d_put:.0f}/{strike_25d_call:.0f} strangle)
      • Target: Collect premium, profit if price stays near ATM.
      • Risk: Large move breaches wings.
      
      Rationale:
      Tail premium excessive. Fade the extremes.
   """)
else:
    print(f"""   
   🦋 STRATEGY 2: Ratio Spread (Skew Arbitrage)
   
      Setup: Skew mismatch → Exploit RR imbalance.
      
      Trade:
      • If PUT SKEW: Buy 1x {strike_25d_put:.0f} Put, Sell 2x {strike_25d_put - 100:.0f} Puts (Risk Reversal)
      • If CALL SKEW: Buy 1x {strike_25d_call:.0f} Call, Sell 2x {strike_25d_call + 100:.0f} Calls
      
      Rationale:
      Harvest skew premium via ratio structures.
   """)

print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
5. RISK WARNINGS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ⚠️  Model Limitations:
   • SABR assumes continuous hedging (real markets have friction).
   • GEX analysis assumes dealers are net short client flow (not always true).
   • Illiquid strikes may have stale/unreliable IVs.
   
   ⚠️  Execution Risks:
   • Wide bid-ask spreads ({futures.spread_bps:.1f} bps on futures).
   • Slippage in options can erode edge.
   • Monitor overnight risk (commodity markets gap).

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   📝 CONCLUSION:
   
   The combination of:
   • {'PUT' if risk_reversal < 0 else 'CALL'} skew (RR = {risk_reversal:+.1f})
   • {'NEGATIVE' if current_gex < 0 else 'POSITIVE'} gamma regime (GEX = {current_gex:.0f})
   • ρ = {sabr_params.rho:+.3f} correlation structure
   
   → Suggests {'HIGH VOLATILITY RISK with downside acceleration potential' if risk_reversal < -1 and current_gex < -100 else 'RANGE-BOUND behavior with pin risk near gamma walls' if current_gex > 100 else 'TRANSITIONAL regime with breakout potential'}.
   
   Position accordingly. Respect the Greeks. Manage size.

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

End of Report.
""")

---

## Appendix: Model Validation

### SABR Calibration Diagnostics

In [ ]:
# Calibration residuals
calib_diagnostics = pd.DataFrame({
    'Strike': calib_calls['strike'].values,
    'Market IV': calib_calls['iv'].values * 100,
    'SABR IV': np.array(model_vols) * 100,
    'Error (%)': (np.array(model_vols) - calib_calls['iv'].values) * 100
})

print("\nSABR Calibration Quality:\n")
print(calib_diagnostics.to_string(index=False))
print(f"\nRMSE: {rmse*100:.3f}%")
print(f"Max Absolute Error: {calib_diagnostics['Error (%)'].abs().max():.3f}%")
print(f"Mean Error: {calib_diagnostics['Error (%)'].mean():.3f}%")

### Close Wind Connection

In [ ]:
# Cleanup
try:
    fetcher.close()
    print("✓ Wind API connection closed")
except:
    pass

print("\n" + "="*80)
print("Analysis Complete. Trade with discipline. Respect the risk.")
print("="*80)